In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [2]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [3]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [4]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [5]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [6]:
df = tripdata[selected_features]

In [7]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 10% NA values

In [8]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc=1):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train, y_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [9]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [10]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train , y_train = ampute_nan_values(X_train,y_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values(X_test,y_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)     
            
    return X_train , y_train, X_test , y_test

In [11]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [12]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [13]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [14]:
model_metrics ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count of varaible  18926
Simulate Imputation for NA
Train after NA value removal Imputation (176642, 21) (176642,)
Test after NA value removal Imputation (44161, 21) (44161,)
Time taken simulate for variable start_lat =  0:00:00.457446
2023-03-25 13:36:38.504550
2023-03-25 13:36:38.803216
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 

sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count of varaible  18926
Simulate Imputation for NA
Train after NA value removal Imputation (176642, 21) (176642,)
Test after NA value removal Imputation (44161, 21) (44161,)
Time taken simulate for variable visibility =  0:00:00.433415
2023-03-25 13:40:10.055029
2023-03-25 13:40:10.277311
Now running model is .... solarradiation
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  75704
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  18926
X_test Amputed value count

In [15]:
print("averge time to run drop is : ", 0.4473875, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=457446),
 datetime.timedelta(microseconds=361567),
 datetime.timedelta(microseconds=694044),
 datetime.timedelta(microseconds=419840),
 datetime.timedelta(microseconds=363392),
 datetime.timedelta(microseconds=402806),
 datetime.timedelta(microseconds=460113),
 datetime.timedelta(microseconds=415425),
 datetime.timedelta(microseconds=460441),
 datetime.timedelta(microseconds=472739),
 datetime.timedelta(microseconds=394760),
 datetime.timedelta(microseconds=419206),
 datetime.timedelta(microseconds=433415),
 datetime.timedelta(microseconds=492359),
 datetime.timedelta(microseconds=437589),
 datetime.timedelta(microseconds=477859),
 datetime.timedelta(microseconds=493361),
 datetime.timedelta(microseconds=405076),
 datetime.timedelta(microseconds=379659),
 datetime.timedelta(microseconds=343389),
 datetime.timedelta(microseconds=610652)]

In [16]:
model_metrics

{'start_lat': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'start_lon': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'end_lat': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'end_lon': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'usertype': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'hour': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'temp': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772015368},
 'feelslike': {'MAE': 1.570092051728064,
  'MSE': 4.998546337663303,
  'RMSE': 2.2357429050906776,
  'R2': 0.5276049772

### Mode Replacement

In [17]:
model_metrics_mode ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.509369
2023-03-25 13:42:53.249799
2023-03-25 13:42:53.592359
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346

sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable conditions =  0:00:00.449215
2023-03-25 13:45:44.078210
2023-03-25 13:45:44.410802
Now model is running for varaible ==== dist
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable dist =  0:00:00.397322
2

In [18]:
print("averge time to run drop is : ", 0.4045785, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=509369),
 datetime.timedelta(microseconds=463130),
 datetime.timedelta(microseconds=373114),
 datetime.timedelta(microseconds=364479),
 datetime.timedelta(microseconds=414422),
 datetime.timedelta(microseconds=484706),
 datetime.timedelta(microseconds=413554),
 datetime.timedelta(microseconds=401809),
 datetime.timedelta(microseconds=339711),
 datetime.timedelta(microseconds=304656),
 datetime.timedelta(microseconds=402726),
 datetime.timedelta(microseconds=407337),
 datetime.timedelta(microseconds=364835),
 datetime.timedelta(microseconds=411258),
 datetime.timedelta(microseconds=449215),
 datetime.timedelta(microseconds=397322),
 datetime.timedelta(microseconds=407737),
 datetime.timedelta(microseconds=389246),
 datetime.timedelta(microseconds=360947),
 datetime.timedelta(microseconds=324529),
 datetime.timedelta(microseconds=512047)]

In [19]:
model_metrics_mode

{'start_lat': {'MAE': 1.5721465611020429,
  'MSE': 4.999929511049258,
  'RMSE': 2.2360522156356852,
  'R2': 0.5272058033111189},
 'start_lon': {'MAE': 1.5734303277634747,
  'MSE': 5.006166054473103,
  'RMSE': 2.2374463243781073,
  'R2': 0.52661607468961},
 'end_lat': {'MAE': 1.5722425217616058,
  'MSE': 5.000655589703993,
  'RMSE': 2.236214567009166,
  'R2': 0.5271371451883312},
 'end_lon': {'MAE': 1.5736280477895215,
  'MSE': 5.003302315729974,
  'RMSE': 2.2368062758607357,
  'R2': 0.5268868703189469},
 'usertype': {'MAE': 1.5803608587891416,
  'MSE': 5.045779589961381,
  'RMSE': 2.2462812802410523,
  'R2': 0.5228702119433858},
 'hour': {'MAE': 1.5742716901763405,
  'MSE': 5.010193500926595,
  'RMSE': 2.238346153061808,
  'R2': 0.5262352386585263},
 'temp': {'MAE': 1.5725853678849735,
  'MSE': 5.000986852309775,
  'RMSE': 2.236288633497424,
  'R2': 0.5271058209392099},
 'feelslike': {'MAE': 1.5724489568661166,
  'MSE': 5.000268340459024,
  'RMSE': 2.2361279794455022,
  'R2': 0.5271737

### Mean Replacement

In [20]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','dew','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

In [21]:
model_metrics_mean ={}
list_comp_time = []
for var in variables_for_mean_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.406142
2023-03-25 13:47:10.932198
2023-03-25 13:47:11.305605
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346

sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable birthyear =  0:00:00.380561
2023-03-25 13:50:03.452097
2023-03-25 13:50:03.832783


In [22]:
print("averge time to run drop is : ", 0.4209185, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=406142),
 datetime.timedelta(microseconds=399258),
 datetime.timedelta(microseconds=395347),
 datetime.timedelta(microseconds=441677),
 datetime.timedelta(microseconds=380364),
 datetime.timedelta(microseconds=484214),
 datetime.timedelta(microseconds=501797),
 datetime.timedelta(microseconds=439095),
 datetime.timedelta(microseconds=403010),
 datetime.timedelta(microseconds=373376),
 datetime.timedelta(microseconds=447084),
 datetime.timedelta(microseconds=472677),
 datetime.timedelta(microseconds=339590),
 datetime.timedelta(microseconds=449586),
 datetime.timedelta(microseconds=380561)]

In [23]:
model_metrics_mean

{'start_lat': {'MAE': 1.5721572333471134,
  'MSE': 4.999910705242945,
  'RMSE': 2.236048010495961,
  'R2': 0.5272075815914063},
 'start_lon': {'MAE': 1.5734004694548605,
  'MSE': 5.005932280703891,
  'RMSE': 2.237394082566567,
  'R2': 0.5266381803775357},
 'end_lat': {'MAE': 1.5722264403256243,
  'MSE': 5.000585645077464,
  'RMSE': 2.2361989278857695,
  'R2': 0.5271437591642762},
 'end_lon': {'MAE': 1.5735868254201535,
  'MSE': 5.003234376150777,
  'RMSE': 2.236791089071748,
  'R2': 0.5268932946972704},
 'hour': {'MAE': 1.5734525860539819,
  'MSE': 5.005085899249781,
  'RMSE': 2.237204930096879,
  'R2': 0.5267182143537756},
 'temp': {'MAE': 1.572568062442452,
  'MSE': 5.000875869431442,
  'RMSE': 2.2362638192823856,
  'R2': 0.5271163154993215},
 'feelslike': {'MAE': 1.572444293322377,
  'MSE': 5.000260285147723,
  'RMSE': 2.236126178270744,
  'R2': 0.5271745252553335},
 'dew': {'MAE': 1.572221528570146,
  'MSE': 4.999426450259082,
  'RMSE': 2.235939724200785,
  'R2': 0.5272533728261792

### Sampling Replacement

In [24]:
model_metrics_sample ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
Time taken simulate for variable start_lat =  0:00:00.035565
2023-03-25 13:50:15.372589
2023-03-25 13:50:15.717288
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
Time taken simulate for variable start_lon =  0:00:00.040019
2023-03-25 13:50:27.150552
2023-03-25 13:50:27.488012
Now model is running for

2023-03-25 13:53:39.790525
2023-03-25 13:53:40.312487
Now model is running for varaible ==== holiday
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
Time taken simulate for variable holiday =  0:00:00.127686
2023-03-25 13:53:53.531126
2023-03-25 13:53:53.950070
Now model is running for varaible ==== day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 75704
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  75704
sample size is = 18926
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  18926
Simple Imputation using sampling
Time taken simulate for variable day =  0:00:00.156553
2023-03-25 13:54:06.513610
2023-03-25 13

In [25]:
print("averge time to run drop is : ", 0.858944, "seconds")
list_comp_time

averge time to run drop is :  0.35481595 seconds


[datetime.timedelta(microseconds=35565),
 datetime.timedelta(microseconds=40019),
 datetime.timedelta(microseconds=60053),
 datetime.timedelta(microseconds=149599),
 datetime.timedelta(microseconds=128193),
 datetime.timedelta(microseconds=69527),
 datetime.timedelta(microseconds=61247),
 datetime.timedelta(microseconds=45595),
 datetime.timedelta(microseconds=41532),
 datetime.timedelta(microseconds=48090),
 datetime.timedelta(microseconds=86437),
 datetime.timedelta(microseconds=59620),
 datetime.timedelta(microseconds=56565),
 datetime.timedelta(microseconds=53755),
 datetime.timedelta(microseconds=130457),
 datetime.timedelta(microseconds=35540),
 datetime.timedelta(microseconds=143420),
 datetime.timedelta(microseconds=127686),
 datetime.timedelta(microseconds=156553),
 datetime.timedelta(microseconds=130633),
 datetime.timedelta(microseconds=143698)]

In [26]:
model_metrics_sample

{'start_lat': {'MAE': 1.5720886041598578,
  'MSE': 5.000219850269955,
  'RMSE': 2.2361171369742587,
  'R2': 0.5271783487843473},
 'start_lon': {'MAE': 1.5739702088576695,
  'MSE': 5.008577217842738,
  'RMSE': 2.2379850798972583,
  'R2': 0.5263880746656271},
 'end_lat': {'MAE': 1.5722364593283198,
  'MSE': 5.001017563407101,
  'RMSE': 2.2362955000194185,
  'R2': 0.5271029168925514},
 'end_lon': {'MAE': 1.5741259999037256,
  'MSE': 5.005285108686993,
  'RMSE': 2.237249451600557,
  'R2': 0.5266993770750436},
 'usertype': {'MAE': 1.5868533509084646,
  'MSE': 5.081024215652425,
  'RMSE': 2.25411273357222,
  'R2': 0.5195374740609069},
 'hour': {'MAE': 1.5743533339872748,
  'MSE': 5.009625737279954,
  'RMSE': 2.2382193228725273,
  'R2': 0.5262889264868507},
 'temp': {'MAE': 1.5726299041934277,
  'MSE': 5.001040252337177,
  'RMSE': 2.2363005728964915,
  'R2': 0.5271007714234112},
 'feelslike': {'MAE': 1.5724451190754378,
  'MSE': 5.000230666638357,
  'RMSE': 2.2361195555332807,
  'R2': 0.52717